<a href="https://www.kaggle.com/code/sejungjenn/spotify-api-with-spotipy-collecting-data?scriptVersionId=115896717" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### How to extract data from spotify using spotify api

In [1]:
!pip install python-dotenv spotipy
!pip install spotipy
import csv
import pandas as pd
import spotipy as spotipy
from spotipy.oauth2 import SpotifyClientCredentials 

In [2]:
client_id = ''
client_secret = ''

client_credentials_manager = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
def get_playlist_track_infos(playlist_link, sp, output_file_name):
    '''Extract track information of each track in the playlist, 
    and create a dataframe with those information'''
    offset = 0
    tracks = []
    playlist_uri = playlist_link.split("/")[-1]
    while True:
        content = sp.playlist_tracks(playlist_uri, fields = None, limit = 100, offset = offset, market = None)
        tracks += content['items']
        # Create a if-else loop so that spotify can extract information more than 100
        if content['next'] is not None:
            offset += 100
        else:
            break
            
    track_info_list = []
    for track in tracks:
        # Track URI
        track_uri = track["track"]["uri"]
        #Track name
        track_name = track["track"]["name"]

        #Main Artist
        artist_name = track["track"]["artists"][0]["name"]

        #Popularity of the track
        track_pop = track["track"]["popularity"]

        track_info_list.append([track_uri, track_name, artist_name, track_pop])

    df = pd.DataFrame(track_info_list, columns = ['track_uri', 'track_name', 'artist', 'track_popularity'])
    df.to_csv("{}.csv".format(output_file_name), index = False)

In [4]:
def get_playlist_audio_features(df, sp, output_file_name):
    '''Extract audio features of each track in specified dataframe, 
    then create a dataframe with those information'''
    track_list = df['track_uri'].to_list()
    features_list = []
    for j in track_list:
        features = sp.audio_features(j)[0]
        features_list.append([j, features['danceability'], features['valence'], features['energy'],
                            features['tempo'], features['loudness'], features['speechiness'], 
                            features['instrumentalness'], features['liveness'], 
                            features['acousticness'], features['key']])

    df = pd.DataFrame(features_list, columns = ['track_uri', 'danceability', 'valence', 'energy',
                                               'tempo', 'loudness', 'speechiness', 
                                                'instrumentalness', 'liveness', 
                                               'acousticness', 'key'])
    df.to_csv("{}.csv".format(output_file_name), index = False)

In [7]:
links = ["https://open.spotify.com/playlist/37i9dQZF1DXe2bobNYDtW8", "https://open.spotify.com/playlist/37i9dQZF1DWVRSukIED0e9", "https://open.spotify.com/playlist/2fmTTbBkXi8pewbUvG3CeZ", "https://open.spotify.com/playlist/5GhQiRkGuqzpWZSE7OU4Se", "https://open.spotify.com/playlist/4hMcqod7ERKJ9mtjgdimeV"]
output_files = ["2018", "2019", "2020", "2021", "2022"]

get_playlist_track_infos(links[0], sp, output_files[0])
df_18 = pd.read_csv('2018.csv')
df_18.head(2)

,track_uri,track_name,artist,track_popularity
0,spotify:track:6DCZcSspjsKoFjzjrWoCdn,God's Plan,Drake,85
1,spotify:track:0e7ipj03S05BNilyu5bRzt,rockstar (feat. 21 Savage),Post Malone,85


### Another option to extract information with limitation of 100 tracks

In [ ]:
!pip install python-dotenv spotipy
!pip install spotipy
import csv
import os
import re
from dotenv import load_dotenv
import pandas as pd
import spotipy as spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import base64
from requests import post 
import json 

In [ ]:
# Authenticate with Spotipy with client_id and client_secret
load_dotenv()

client_id = os.getenv("CLIENT_ID","0624d92123024fd4a7b2017c8d531c94") # tyoe in your client_id in between the quotation marks
client_secret = os.getenv("CLIENT_SECRET", "4127bf4ec5cf4ab28140a7bba2036ac9") # input your client_secret in between the quotation marks

client_credentials_manager = SpotifyClientCredentials(client_id = client_id, client_secret= client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
links = ["https://open.spotify.com/playlist/37i9dQZF1DXe2bobNYDtW8", "https://open.spotify.com/playlist/37i9dQZF1DWVRSukIED0e9", "https://open.spotify.com/playlist/2fmTTbBkXi8pewbUvG3CeZ", "https://open.spotify.com/playlist/5GhQiRkGuqzpWZSE7OU4Se", "https://open.spotify.com/playlist/4hMcqod7ERKJ9mtjgdimeV"]
output_files = ["2018.csv", "2019.csv", "2020.csv", "2021.csv", "2022.csv"]

for link, output_file in zip(links, output_files):
    with open(output_file, "w", encoding = 'utf-8') as file:
            uris = link.split("/")[-1]
            writer = csv.writer(file)
            writer.writerow(['track_uri', 'track', 'artist', 'artist_popularity', 'followers', 'artist_genre', 'track_popularity', 'album'])
            for track in sp.playlist_tracks(uris)["items"]:
                #URI
                track_uri = track["track"]["uri"]

                #Track name
                track_name = track["track"]["name"]

                #Main Artist
                artist_uri = track["track"]["artists"][0]["uri"]
                artist_info = sp.artist(artist_uri)

                #Name, popularity, genre
                artist_name = track["track"]["artists"][0]["name"]
                artist_pop = artist_info["popularity"]
                artist_genre = artist_info["genres"]
                followers = artist_info["followers"]['total']

                #Album
                album = track["track"]["album"]["name"]

                #Popularity of the track
                track_pop = track["track"]["popularity"]

In [ ]:
links = ["https://open.spotify.com/playlist/37i9dQZF1DXe2bobNYDtW8", "https://open.spotify.com/playlist/37i9dQZF1DWVRSukIED0e9", "https://open.spotify.com/playlist/2fmTTbBkXi8pewbUvG3CeZ", "https://open.spotify.com/playlist/5GhQiRkGuqzpWZSE7OU4Se", "https://open.spotify.com/playlist/4hMcqod7ERKJ9mtjgdimeV"]
output_files = ["2018.csv", "2019.csv", "2020.csv", "2021.csv", "2022.csv"]

for link, output_file in zip(links, output_files):
    with open(output_file, "w", encoding = 'utf-8') as file:
            uris = link.split("/")[-1]
            writer = csv.writer(file)
            writer.writerow(['track_uri', 'track', 'artist', 'artist_popularity', 'followers', 'artist_genre', 'track_popularity', 'album'])
            for track in sp.playlist_tracks(uris)["items"]:
                #URI
                track_uri = track["track"]["uri"]

                #Track name
                track_name = track["track"]["name"]

                #Main Artist
                artist_uri = track["track"]["artists"][0]["uri"]
                artist_info = sp.artist(artist_uri)

                #Name, popularity, genre
                artist_name = track["track"]["artists"][0]["name"]
                artist_pop = artist_info["popularity"]
                artist_genre = artist_info["genres"]
                followers = artist_info["followers"]['total']

                #Album
                album = track["track"]["album"]["name"]

                #Popularity of the track
                track_pop = track["track"]["popularity"]

                # write a csv file
                writer.writerow([track_uri, track_name, artist_name, artist_pop, followers, artist_genre, track_pop, album])

In [ ]:
df2018 = pd.read_csv("2018.csv")
df2019 = pd.read_csv("2019.csv")
df2020 = pd.read_csv("2020.csv")
df2021 = pd.read_csv("2021.csv")
df2022 = pd.read_csv('2022.csv')
df2018.head()

In [ ]:
# Add year column to each table
df2018['year'] = 2018
df2019['year'] = 2019
df2020['year'] = 2020
df2021['year'] = 2021
df2022['year'] = 2022

In [ ]:
# Confirm
df2018.info()

In [ ]:
df2020.head(1)

In [ ]:
df2021.head(1)

In [ ]:
df2022.head(1)

In [ ]:
def uri_column(data):
    new = data['track_uri'].str.split(":", expand = True)
    data['track_uri'] = new[2].astype(str)
    
uri_column(df2018)
uri_column(df2019)
uri_column(df2020)
uri_column(df2021)
uri_column(df2022)
df2018.head(1)

In [ ]:
def get_token():
    '''Get token'''
    auth_string = client_id + ":" + client_secret 
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), 'utf-8')

    url = "https://accounts.spotify.com/api/token"

    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded",
    }

    data = {'grant_type': "client_credentials"}
    result = post(url, headers = headers, data = data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

token = get_token()

headers = {'Authorization': 'Bearer {}'.format(token)}

In [ ]:
base_url = 'https://api.spotify.com/v1/'
df_lists = [df2018, df2019, df2020, df2021, df2022]
output_files = ['track2018.csv', 'track2019.csv', 'track2020.csv', 'track2021.csv', 'track2022.csv']

for df, output_file in zip(df_lists, output_files):
    with open(output_file, "w", encoding = 'utf-8') as file:
        uris = df['track_uri'].to_list()
        features = {}
        writer = csv.writer(file)
        writer.writerow(["track_uri", "danceability", "valence", "energy", "tempo", "loudness", "speechiness", "instrumentalness", "liveness", "acousticness", "key"])
        for uri in uris:
            track_uri = uri
            
            features[uri] = {
                            'danceability': 0,
                            'valence': 0,
                            'energy': 0,
                            'tempo': 0,
                            'loudness': 0,
                            'speechiness': 0, 
                            'instrumentalness': 0,
                            'liveness': 0, 
                            'acousticness': 0, 
                            'key': 0
                            }

            s = requests.get(base_url + 'audio-features/' + uri, headers = headers)
            s = s.json()
            
            features[uri]['danceability'] = s['danceability']
            features[uri]['valence'] = s['valence']
            features[uri]['energy'] = s['energy']
            features[uri]['tempo'] = s['tempo']
            features[uri]['loudness'] = s['loudness']
            features[uri]['speechiness'] = s['speechiness']
            features[uri]['instrumentalness'] = s['instrumentalness']
            features[uri]['liveness'] = s['liveness']
            features[uri]['acousticness'] = s['acousticness']
            features[uri]['key'] = s['key']
            
            writer.writerow([track_uri, features[uri]['danceability'], features[uri]['valence'], features[uri]['energy'], features[uri]['tempo'], features[uri]['loudness'], features[uri]['speechiness'], 
                            features[uri]['instrumentalness'], features[uri]['liveness'], features[uri]['acousticness'], features[uri]['key']])

In [ ]:
df2_2018 = pd.read_csv("track2018.csv")
df2_2019 = pd.read_csv("track2019.csv")
df2_2020 = pd.read_csv("track2020.csv")
df2_2021 = pd.read_csv("track2021.csv")
df2_2022 = pd.read_csv("track2022.csv")

In [ ]:
df2018 = pd.merge(df2018, df2_2018, how = 'inner', on = 'track_uri')
df2019 = pd.merge(df2019, df2_2019, how = 'inner', on = 'track_uri')
df2020 = pd.merge(df2020, df2_2020, how = 'inner', on = 'track_uri')
df2021 = pd.merge(df2021, df2_2021, how = 'inner', on = 'track_uri')
df2022 = pd.merge(df2022, df2_2022, how = 'inner', on = 'track_uri')

In [ ]:
df2018.to_csv("df2018.csv", index = False)
df2019.to_csv("df2019.csv", index = False)
df2020.to_csv("df2020.csv", index = False)
df2021.to_csv("df2021.csv", index = False)
df2022.to_csv("df2022.csv", index = False)

In [ ]:
df_complete = pd.concat([df2018, df2019, df2020,  df2021, df2022], ignore_index = True)
df_complete.to_csv("df_complete.csv", index = False)